<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series
Timestamps, períodos, intervalos

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

# Operações com datas

In [ ]:
from datetime import datetime
agora = datetime.now()
utc = datetime.utcnow()

agora.year
agora.month
agora.day
agora

datetime.datetime(2021, 8, 11, 14, 6, 8, 731871)

In [ ]:
# aterar hora
agora.replace(minute=10, hour=10, second=10)

datetime.datetime(2021, 8, 11, 10, 10, 10, 731871)

In [ ]:
# definir uma data
evento = datetime(2021, 8, 1)
evento

datetime.datetime(2021, 8, 1, 0, 0)

In [ ]:
# diferença entre datas retorna um obj do tipo timedelta
delta = agora - evento
print(delta)
print(f"Desde o evento já se passaram {delta.days} dias e {delta.seconds // 3600} horas.")

10 days, 14:06:08.731871
Desde o evento já se passaram 10 dias e 14 horas.


In [ ]:
# soma retorna uma nova data e uma subtração retorna um delta
evento + delta

datetime.datetime(2021, 8, 11, 14, 6, 8, 731871)

In [ ]:
# dia da semana 
# usar tupla porque essa lista é imutavel
dias = ("Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sabado", "Domingo")

print(dias[agora.weekday()])

print(f"Dia que nasci: {dias[datetime(day=14, month=10, year=1985).weekday()]}")

Quarta
Dia que nasci: Segunda


# Conversão string e datetime

### strftime e strptime

In [ ]:
# data para > string
print(f"Hoje (sem formatação): {agora}")

hoje_formatado = agora.strftime("%d/%m/%Y")
print(f"Hoje (formatado dd/mm/aaaa): {hoje_formatado}")

hoje_formatado = agora.strftime("%m/%d/%y")
print(f"Hoje (formatado mm/dd/aa): {hoje_formatado}")

hoje_formatado = agora.strftime("%d/%B/%y")
print(f"Hoje (formatado): {hoje_formatado}")

Hoje (sem formatação): 2021-08-11 14:06:08.731871
Hoje (formatado dd/mm/aaaa): 11/08/2021
Hoje (formatado mm/dd/aa): 08/11/21
Hoje (formatado): 11/August/21


In [ ]:
# String para > data. É diferente de strftime(), strftime é método do objeto e strptime é metodo da classe datetime
nascimento = datetime.strptime("10/31/2021", "%m/%d/%Y")
print(nascimento)
print(type(nascimento))

2021-10-31 00:00:00
<class 'datetime.datetime'>


In [ ]:
# converte uma lista de datas no formato str
datestrs = ['7/6/2011', '8/6/2011', '18/10/2021']
datestrs = [datetime.strptime(dt, "%d/%m/%Y") for dt in datestrs]
datestrs

[datetime.datetime(2011, 6, 7, 0, 0),
 datetime.datetime(2011, 6, 8, 0, 0),
 datetime.datetime(2021, 10, 18, 0, 0)]

### parse

In [ ]:
# Parse: uma alternativa para simplificar o mapeamento das strs para datas sem passar o formato
from dateutil.parser import parse

# faz a conversão mesmo com tipos de formatos diferentes
datesparse = ['7/06/2011', '8/16/2011', '18/10/2021', 'Jan 31, 1997 10:45 PM', None]

# daysfirst informa o padrão de reconhecimento
# mesmo que o dia esteja no lugar do mês ele reconhece que é maior que 12 e trata como dia
# precisa tratar os valores None senão retorna erro
datesparse = [parse(dt, dayfirst=True) for dt in datesparse if dt != None]
datesparse

[datetime.datetime(2011, 6, 7, 0, 0),
 datetime.datetime(2011, 8, 16, 0, 0),
 datetime.datetime(2021, 10, 18, 0, 0),
 datetime.datetime(1997, 1, 31, 22, 45)]

### to_datetime

In [ ]:
# o parse com to_datetime funciona igual, mas trata os erros, inserie como NaT (Not a Time)
# retorna um obj do tipo DatetimeIndex com algumas propriedades adicionais 
dates_todatetime = ['2/01/2011', '7/06/2011', '8/16/2011', '8/16/2011', '18/10/2021', 'Jan 31, 1997 10:45 PM', None]
dates_todatetime = pd.to_datetime(dates_todatetime)

dates_todatetime.name = "Indice_DateTime"

# propriedades adicionais de um indice
print(dates_todatetime)
print("---------------------")
print(type(dates_todatetime))
print(dates_todatetime.is_unique)
print(dates_todatetime.name)
print("---------------------")
print(pd.isnull(dates_todatetime))
print("---------------------")

# tratando o ultimo valor que é NaT com fillna
dates_todatetime = dates_todatetime.fillna(datetime.now())
dates_todatetime

DatetimeIndex(['2011-02-01 00:00:00', '2011-07-06 00:00:00',
               '2011-08-16 00:00:00', '2011-08-16 00:00:00',
               '2021-10-18 00:00:00', '1997-01-31 22:45:00',
                               'NaT'],
              dtype='datetime64[ns]', name='Indice_DateTime', freq=None)
---------------------
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
False
Indice_DateTime
---------------------
[False False False False False False  True]
---------------------


DatetimeIndex([       '2011-02-01 00:00:00',        '2011-07-06 00:00:00',
                      '2011-08-16 00:00:00',        '2011-08-16 00:00:00',
                      '2021-10-18 00:00:00',        '1997-01-31 22:45:00',
               '2021-08-11 14:43:06.047138'],
              dtype='datetime64[ns]', name='Indice_DateTime', freq=None)

# Series Temporais
São series indexadas por datas

In [ ]:
dates_list = [datetime(2011, 2, 1), datetime(2011, 1, 5), datetime(2011, 1, 7), datetime(2011, 1, 8), datetime(2011, 1, 10), datetime(2011, 1, 12)]

# series indexada por datas
time_series = pd.Series(np.random.randn(6), index=dates_list, name="time_series")
time_series.index

DatetimeIndex(['2011-02-01', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
# criando um novo ts usando a lista de dadas criadas anteriormente.
time_series2 = pd.Series(np.random.randn(7), index=dates_todatetime, name="time_series2")
time_series2.index

DatetimeIndex([       '2011-02-01 00:00:00',        '2011-07-06 00:00:00',
                      '2011-08-16 00:00:00',        '2011-08-16 00:00:00',
                      '2021-10-18 00:00:00',        '1997-01-31 22:45:00',
               '2021-08-11 14:43:06.047138'],
              dtype='datetime64[ns]', name='Indice_DateTime', freq=None)

### operações entre series indexadas por datas

In [ ]:
# operações entre series indexadas por datas
# ao fazer o alinhamento entre series pelo indice

# pareamento das chaves e o calculo de soma
ts_soma = time_series + time_series2

# no dia 01/02/2011 existe uma correspondencia entre as chaves
ts_soma.dropna()
#ts_soma[ts_soma.notnull()]

2011-02-01   -2.314266
dtype: float64

## Filtrar series temporais (slice)

In [ ]:
time_series

2011-02-01   -0.010032
2011-01-05    0.050009
2011-01-07    0.670216
2011-01-08    0.852965
2011-01-10   -0.955869
2011-01-12   -0.023493
Name: time_series, dtype: float64

In [ ]:
# Podemos usar uma string interpretável
# Filtrar apenas as datas maiores que 2011-01-07
time_series['2011-01-07':]

2011-01-07    0.670216
2011-01-08    0.852965
2011-01-10   -0.955869
2011-01-12   -0.023493
Name: time_series, dtype: float64

In [ ]:
# Filtrar dois dias específicos
time_series[['2011-01-07', '2011-01-12']]

2011-01-07    0.670216
2011-01-12   -0.023493
Name: time_series, dtype: float64

In [ ]:
# Filtrar usando um indice sequencial implicito
time_series[2:4]

2011-01-07    0.670216
2011-01-08    0.852965
Name: time_series, dtype: float64

In [ ]:
# em series temporais mais longas um ano ou um mês podem ser usado como filtro de período

# criar uma serie longa
idx_datarange = pd.date_range(start="2020-01-01", end="2021-12-31")
time_series_grande = pd.Series(np.arange(len(idx_datarange)), index=idx_datarange)
time_series_grande

2020-01-01      0
2020-01-02      1
2020-01-03      2
2020-01-04      3
2020-01-05      4
             ... 
2021-12-27    726
2021-12-28    727
2021-12-29    728
2021-12-30    729
2021-12-31    730
Freq: D, Length: 731, dtype: int64

In [ ]:
# filtar todos os dias do ano 2021
time_series_grande["2021"]

2021-01-01    366
2021-01-02    367
2021-01-03    368
2021-01-04    369
2021-01-05    370
             ... 
2021-12-27    726
2021-12-28    727
2021-12-29    728
2021-12-30    729
2021-12-31    730
Freq: D, Length: 365, dtype: int64

In [ ]:
# filtrar um mês do ano
time_series_grande["2021-12"]

2021-12-01    700
2021-12-02    701
2021-12-03    702
2021-12-04    703
2021-12-05    704
             ... 
2021-12-27    726
2021-12-28    727
2021-12-29    728
2021-12-30    729
2021-12-31    730
Freq: D, Length: 31, dtype: int64

In [ ]:
# filtrar um periodo
time_series_grande["2021-12-01": "2021-12-05"]

2021-12-01    700
2021-12-02    701
2021-12-03    702
2021-12-04    703
2021-12-05    704
Freq: D, dtype: int64

In [ ]:
# remove todas as linhas "depois" de 2020-12-31
time_series_grande.truncate(after="2020-12-31")

2020-01-01      0
2020-01-02      1
2020-01-03      2
2020-01-04      3
2020-01-05      4
             ... 
2020-12-27    361
2020-12-28    362
2020-12-29    363
2020-12-30    364
2020-12-31    365
Freq: D, Length: 366, dtype: int64

## Indices de data duplicados

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [ ]:
# possiu indice duplicado
dup_ts.index.is_unique

False

In [ ]:
# agregação para identificar o grau de duplicação para tratar ou agregar
dup_ts.groupby(level=0).count()

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [88]:
# atualiza a series apenas com os valores agregados. Agrega os duplicados usando a média 
dup_ts = dup_ts.groupby(level=0).mean()

print(f"O índice é unico? {dup_ts.is_unique}\n")

dup_ts

O índice é unico? True



2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

## Ranges de Datas

In [89]:
# criando um range a partir de uma data inicial. As datas avançam
pd.date_range(start='2012-04-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [90]:
# criando um range de datas a partir da data final. As datas voltam
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [98]:
# Criando um range usando um período. freq="D" uma data por dia no período
date_index = pd.date_range(start="2020-01-01", end="2021-12-31", freq="D")
date_index

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=731, freq='D')

In [99]:
# altera da freq="BM", a ultima data de cada mês do período.
date_index = pd.date_range(start="2020-01-01", end="2021-12-31", freq="BM")
date_index

DatetimeIndex(['2020-01-31', '2020-02-28', '2020-03-31', '2020-04-30',
               '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-30', '2020-11-30', '2020-12-31',
               '2021-01-29', '2021-02-26', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-30', '2021-08-31',
               '2021-09-30', '2021-10-29', '2021-11-30', '2021-12-31'],
              dtype='datetime64[ns]', freq='BM')

In [136]:
# normalize=True remove a parte do horário das datas que tenham horário
date_index = pd.date_range(start=datetime.now(), periods=20, normalize=True)
date_index

DatetimeIndex(['2021-08-11', '2021-08-12', '2021-08-13', '2021-08-14',
               '2021-08-15', '2021-08-16', '2021-08-17', '2021-08-18',
               '2021-08-19', '2021-08-20', '2021-08-21', '2021-08-22',
               '2021-08-23', '2021-08-24', '2021-08-25', '2021-08-26',
               '2021-08-27', '2021-08-28', '2021-08-29', '2021-08-30'],
              dtype='datetime64[ns]', freq='D')

In [137]:
# a cada 1h30min
date_index = pd.date_range('2000-01-01', periods=10, freq='1h30min')
date_index

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

## Time Zone

In [132]:
import pytz
# pytz.common_timezones

# lista de timezones ultimos 5
print("US/Eastern" in pytz.common_timezones[-5:])

print(pytz.common_timezones[-5:])

True
['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']


In [124]:
# brasil
[tz for tz in pytz.common_timezones if tz.startswith("America/Sao_Paulo")]

['America/Sao_Paulo']

In [141]:
# timezone por padrão é None
print(date_index.tz)

None


In [144]:
# timezone UTC é o central +0 qualquer mundança adiciona ou subtrai
date_index = pd.date_range('2000-01-01', periods=10, freq='1h30min', tz="UTC")
date_index

DatetimeIndex(['2000-01-01 00:00:00+00:00', '2000-01-01 01:30:00+00:00',
               '2000-01-01 03:00:00+00:00', '2000-01-01 04:30:00+00:00',
               '2000-01-01 06:00:00+00:00', '2000-01-01 07:30:00+00:00',
               '2000-01-01 09:00:00+00:00', '2000-01-01 10:30:00+00:00',
               '2000-01-01 12:00:00+00:00', '2000-01-01 13:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='90T')

In [148]:
# timezone Brasil é o central +2
date_index = pd.date_range('2000-01-01', periods=10, freq='1h30min', tz="America/Sao_Paulo")
date_index

DatetimeIndex(['2000-01-01 00:00:00-02:00', '2000-01-01 01:30:00-02:00',
               '2000-01-01 03:00:00-02:00', '2000-01-01 04:30:00-02:00',
               '2000-01-01 06:00:00-02:00', '2000-01-01 07:30:00-02:00',
               '2000-01-01 09:00:00-02:00', '2000-01-01 10:30:00-02:00',
               '2000-01-01 12:00:00-02:00', '2000-01-01 13:30:00-02:00'],
              dtype='datetime64[ns, America/Sao_Paulo]', freq='90T')

In [149]:
# converte de uma timezone para outra região (Brasil > Nova York)
# -5 horas timezone
date_index.tz_convert("America/New_York")

DatetimeIndex(['1999-12-31 21:00:00-05:00', '1999-12-31 22:30:00-05:00',
               '2000-01-01 00:00:00-05:00', '2000-01-01 01:30:00-05:00',
               '2000-01-01 03:00:00-05:00', '2000-01-01 04:30:00-05:00',
               '2000-01-01 06:00:00-05:00', '2000-01-01 07:30:00-05:00',
               '2000-01-01 09:00:00-05:00', '2000-01-01 10:30:00-05:00'],
              dtype='datetime64[ns, America/New_York]', freq='90T')

## Timestamp

In [150]:
# criando localizando e convertendo para outro utc
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

# Reamostragem e conversão de frequência
Transformar uma serie temporal que está em uma frequencia para outra
- downsampling: frequencias mais altas para mais baixas
- upsampling: 

## Downsampling: resample()

In [154]:
# frequencia diaria
data_range_resample = pd.date_range('2000-01-01', periods=100, freq='D')

ts = pd.Series(np.random.randn(len(data_range_resample)), index=data_range_resample)

# semelhante a groupby. Agrupa no nivel de mes e aplica uma agregação mean
ts.resample('M').mean()

# ts.resample('M', kind='period').mean()


2000-01-31    0.118426
2000-02-29   -0.143746
2000-03-31    0.021168
2000-04-30    0.736457
Freq: M, dtype: float64